# Análise de Tópicos - BERTopic



In [ ]:
from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups
import pandas as pd
import re
import torch
import itertools
import nltk


#Importando bibliotecas
import re
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from os import path
import matplotlib.pyplot as plt

In [ ]:
df_topic = pd.read_csv('df_final.csv')

In [ ]:
nltk.download('stopwords',quiet=True)
nltk.download('averaged_perceptron_tagger',quiet=True)
nltk.download('wordnet',quiet=True)
nltk.download('vader_lexicon',quiet=True)
nltk.download('punkt', quiet = True)

In [ ]:
def is_vow(c):
    return ((c == 'a') or (c == 'e') or 
            (c == 'i') or (c == 'o') or 
            (c == 'u'));
  
# function to print resultant string
def removeVowels(str):
    result = []
  
    try:
        result.append(str[0])
    except:
        pass
  
    # loop to check for each character
    for i in range(1,len(str)):
  
        # comparison of consecutive
        # characters
            
        if ((str[i-1] == str[i]) and (is_vow(str[i-1]) == True) and (is_vow(str[i]) == True)):
            pass
        else:
            result.append(str[i])
            #print(str[i])
            
    str1 = ''.join(result)
    return str1

def corrige_internetes(sentenca):

    sentenca = re.sub(r'^RT[\s]+', '', str(sentenca))
    sentenca = re.sub(r"(?:\@|https?\://)\S+", "", str(sentenca))
    sentenca = re.sub("4ogers", " ", str(sentenca))
    sentenca = re.sub("distribuindo", " ", str(sentenca))
    sentenca = re.sub("distribuição", " ", str(sentenca))
    sentenca = re.sub("dist", " ", str(sentenca))
    sentenca = re.sub("\n"," ", sentenca)
    sentenca = removeVowels(sentenca)

    return sentenca

def clean_text(text):
    import string
    from nltk.corpus import stopwords
    from nltk import pos_tag
    from nltk.tokenize import WhitespaceTokenizer
    from nltk.stem import WordNetLemmatizer
    
    #Passa todos os caracteres do texto para minúsculo.
    text = text.lower()
    
    #As pontuações foram removidas e os textos foram separados por palavras.
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    
    #Remove palavras que contém números
    text = [word for word in text if not any(c.isdigit() for c in word)]
    
    #Remove stopwords como the, a, that, dentre outras.
    stop = stopwords.words('portuguese')
    text = [x for x in text if x not in stop]
    
    #Remove tokens vazios.
    text = [t for t in text if len(t) > 0]
    
    #Faz a lematização dos textos, transformando as palavras em sua forma original.
    #text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
    
    #Remove palavras com apenas uma letra.
    text = [t for t in text if len(t) > 1]
    
    #Junta tudo de volta.
    text = " ".join(text)
    return(text)

In [ ]:
df_topic = df_topic[df_topic['Polaridade'].notna()]

In [ ]:
df_topic

In [ ]:
df_topic["Mensagem"] = df_topic["Mensagem"].apply(lambda x: str(x).lower())
df_topic["Mensagem"] = df_topic["Mensagem"].apply(lambda x: corrige_internetes(str(x)))
df_topic["Mensagem"] = df_topic["Mensagem"].apply(lambda x: clean_text(str(x)))

In [ ]:
bolsonaro = df_topic[df_topic["candidato"] == 'bolsonaro']
bolsonaro_positivo = bolsonaro[bolsonaro["Polaridade"] == 'Positiva']
bolsonaro_negativo = bolsonaro[bolsonaro["Polaridade"] == 'Negativa']

haddad = df_topic[df_topic["candidato"] == 'haddad']
haddad_positivo = haddad[haddad["Polaridade"] == 'Positiva']
haddad_negativo = haddad[haddad["Polaridade"] == 'Negativa']

In [ ]:
haddad_positivo

In [ ]:
haddad_negativo

In [ ]:
bolsonaro_positivo

In [ ]:
bolsonaro_negativo

In [ ]:
list_bolsoneg = bolsonaro_negativo['Mensagem'].tolist()
list_bolsopos = bolsonaro_positivo['Mensagem'].tolist()

list_haddadneg = haddad_negativo['Mensagem'].tolist()
list_haddadpos = haddad_positivo['Mensagem'].tolist()

In [ ]:
topic_model_bolsopos = BERTopic(embedding_model='neuralmind/bert-large-portuguese-cased', nr_topics=20)

In [ ]:
topic_model_bolsoneg = BERTopic(embedding_model='neuralmind/bert-large-portuguese-cased', nr_topics=20)

In [ ]:
topic_model_haddpos = BERTopic(embedding_model='neuralmind/bert-large-portuguese-cased', nr_topics=20)

In [ ]:
topic_model_haddneg = BERTopic(embedding_model='neuralmind/bert-large-portuguese-cased', nr_topics=20)

In [ ]:
topics_bolsopos, probs_bolsopos = topic_model_bolsopos.fit_transform(list_bolsopos)

In [ ]:
topics_bolsoneg, probs_bolsoneg = topic_model_bolsoneg.fit_transform(list_bolsoneg)

In [ ]:
topics_haddpos, probs_haddpos = topic_model_haddpos.fit_transform(list_haddadpos)

In [ ]:
topics_haddneg, probs_haddneg = topic_model_haddneg.fit_transform(list_haddadneg)

In [ ]:
topic_model_bolsopos.visualize_topics()

In [ ]:
topic_model_bolsoneg.visualize_topics()

In [ ]:
topic_model_bolsopos.get_topic_info()

In [ ]:
topic_model_bolsoneg.visualize_barchart(top_n_topics=9)

In [ ]:
#df_bolsoneg = df = pd.DataFrame()
#for i in range(946):
#    aux = pd.DataFrame(topic_model_bolsoneg.get_topic(i), columns=[["topico", "prob"]])
#    aux['topic_num'] = i
#    df_bolsoneg = pd.concat([df_bolsoneg, aux])
#    
#df_bolsoneg.reset_index(inplace=True)

In [ ]:
#df_bolsopos = df = pd.DataFrame()
#for i in range(1585):
#    aux = pd.DataFrame(topic_model_bolsopos.get_topic(i), columns=[["topico", "prob"]])
#    aux['topic_num'] = i
#    df_bolsopos = pd.concat([df_bolsopos, aux])
#    
#df_bolsopos.reset_index(inplace=True)

In [ ]:
#df_haddpos = df = pd.DataFrame()
#for i in range(1300):
#    aux = pd.DataFrame(topic_model_haddpos.get_topic(i), columns=[["topico", "prob"]])
#    aux['topic_num'] = i
#    df_haddpos = pd.concat([df_haddpos, aux])
#    
#df_haddpos.reset_index(inplace=True)

In [ ]:
#df_haddneg = df = pd.DataFrame()
#for i in range(511):
#    aux = pd.DataFrame(topic_model_haddneg.get_topic(i), columns=[["topico", "prob"]])
#    aux['topic_num'] = i
#    df_haddneg = pd.concat([df_haddneg, aux])
#    
#df_haddneg.reset_index(inplace=True)

In [ ]:
#df_bolsoneg.to_excel('bolsonaro_topicos_negativos.xlsx')
#df_bolsopos.to_excel('bolsonaro_topicos_positivos.xlsx')
#df_haddneg.to_excel('haddad_topicos_negativos.xlsx')
#df_haddpos.to_excel('haddad_topicos_positivos.xlsx')